## Class06

In [1]:
import pandas as pd
import numpy as np

### Execucomp

In [2]:
file_path = '/Users/ml/Google Drive/af/teaching/database/data/'
execucomp_raw = pd.read_csv(file_path+'execucomp_1992_2017.txt',sep='\t',engine='python')
execucomp_raw.columns = execucomp_raw.columns.str.lower()

In [3]:
execucomp_raw.head()

,exec_fullname,cfoann,execdir,co_per_rol,coname,ceoann,salary,total_sec,shrown_excl_opts_pct,age,...,becameceo,joined_co,leftofc,leftco,pceo,pcfo,gender,page,cusip,sic
0,Ira A. Eichner,NaN,1,5622,AAR CORP,NaN,460.4,NaN,2.7,NaN,...,19550101.0,NaN,19961001.0,19990531.0,NaN,NaN,MALE,87.0,00036110,5080
1,David P. Storch,NaN,1,5623,AAR CORP,NaN,322.3,NaN,NaN,NaN,...,19961009.0,NaN,20180531.0,NaN,CEO,NaN,MALE,65.0,00036110,5080
2,Philip C. Slapke,NaN,0,5624,AAR CORP,NaN,205.9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,MALE,NaN,00036110,5080
3,Howard A. Pulsifer,NaN,0,5625,AAR CORP,NaN,159.1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,MALE,75.0,00036110,5080
4,Timothy J. Romenesko,NaN,0,5626,AAR CORP,NaN,109.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,CFO,MALE,61.0,00036110,5080


#### Check duplicates

In [4]:
execucomp = execucomp_raw.copy()
execucomp = execucomp.sort_values(['gvkey','year','execid']).reset_index(drop=True)
execucomp = execucomp.drop_duplicates(['gvkey','year','execid'])
execucomp = execucomp.sort_values(['gvkey','year']).reset_index(drop=True)

#### Historical CEO
**ceoann** is the indicator to track historical CEO. **pceo** is the indicator to identify current CEO.

In [5]:
ceo = execucomp[execucomp['ceoann']=='CEO'].copy()
ceo = ceo.sort_values(['gvkey','year']).reset_index(drop=True)

#### Historical CFO
**cfoann** is the indicator to track historical CFO. **pcfo** is the indicator to identify current CFO.

In [6]:
cfo = execucomp[execucomp['cfoann']=='CFO'].copy()
cfo = cfo.sort_values(['gvkey','year']).reset_index(drop=True)

#### Male vs female executives

In [7]:
execucomp['measure'] = 'percent'
pd.crosstab(execucomp['measure'],execucomp['gender']).apply(lambda x: x/x.sum(),axis=1).T \
    .style.format('{:.2%}')

measure,percent
gender,
FEMALE,6.02%
MALE,93.98%


#### Male vs female CEO

In [8]:
ceo['measure'] = 'percent'
pd.crosstab(ceo['measure'],ceo['gender']).apply(lambda x: x/x.sum(),axis=1).T.style.format('{:.2%}')

measure,percent
gender,
FEMALE,2.22%
MALE,97.78%


#### CEO tenure

In [9]:
ceo['tenure'] = ceo.groupby(['gvkey','execid'])['year'].cumcount() + 1
ceo[['gvkey','year','exec_fullname','tenure']].head(10)

,gvkey,year,exec_fullname,tenure
0,1004,1994,Ira A. Eichner,1
1,1004,1995,Ira A. Eichner,2
2,1004,1996,David P. Storch,1
3,1004,1997,David P. Storch,2
4,1004,1998,David P. Storch,3
5,1004,1999,David P. Storch,4
6,1004,2000,David P. Storch,5
7,1004,2001,David P. Storch,6
8,1004,2002,David P. Storch,7
9,1004,2003,David P. Storch,8


#### Number of executive by firm-year

In [10]:
n_exec = execucomp.groupby(['gvkey','year'])['execid'].count().to_frame('n_exec')

#### Percent of female executives by firm-year

In [11]:
pct_female = execucomp.groupby(['gvkey','year','gender'])['execid'].count().unstack().reset_index()
pct_female['FEMALE'] = np.where(pct_female['FEMALE'].isnull(),0,pct_female['FEMALE'])
pct_female['pct_female'] = pct_female['FEMALE'] / (pct_female['FEMALE']+pct_female['MALE'])

#### CEO ownership and top5 executives ownership

In [12]:
execucomp['shrown_excl_opts_pct'] = execucomp['shrown_excl_opts_pct'] / 100
ceoown = execucomp[execucomp['ceoann']=='CEO'].groupby(['gvkey','year'])['shrown_excl_opts_pct'] \
    .sum(min_count=1).to_frame('ceoown').reset_index()
top5own = execucomp[execucomp['execrankann']<=5].groupby(['gvkey','year'])['shrown_excl_opts_pct'] \
    .sum(min_count=1).to_frame('top5own').reset_index()

In [13]:
execucomp_1 = pct_female.merge(ceoown,how='left',on=['gvkey','year']).reset_index()
execucomp_1 = execucomp_1.sort_values(['gvkey','year']).reset_index(drop=True)

### ROA

In [14]:
roa_raw = pd.read_csv(file_path+'roa.txt',sep='\t')

In [15]:
roa = roa_raw.copy()
roa['calyr'] = (roa['datadate']/10000).astype(int)
roa = roa.sort_values(['gvkey','datadate']).reset_index(drop=True)
roa = roa.drop_duplicates(['gvkey','calyr'],keep='last')
roa = roa.sort_values(['gvkey','calyr']).reset_index(drop=True)
roa['at_l1'] = roa.groupby('gvkey')['at'].shift(1)
roa['seq_l1'] = roa.groupby('gvkey')['seq'].shift(1)

#### ROA

In [16]:
roa['roa'] = roa['ib'] / roa['at_l1']

#### ROE

In [17]:
roa['roe'] = roa['ib'] / roa['seq_l1']

#### Leverage

In [18]:
roa['lev'] = roa['lt'] / roa['at']

In [19]:
for i in ['roa','roe','lev']:
    roa[i] = np.where((roa[i]==np.inf)|(roa[i]==-np.inf),np.nan,roa[i])

### Regression

In [20]:
import statsmodels.formula.api as sm
from linearmodels import PanelOLS

In [21]:
execucomp_2 = execucomp_1.merge(roa[['gvkey','fyear','roa','roe','lev']],how='inner', \
    left_on=['gvkey','year'],right_on=['gvkey','fyear'])

#### OLS

In [22]:
sm.ols('roa~pct_female+ceoown',execucomp_2).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    roa   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     16.90
Date:                Thu, 16 Aug 2018   Prob (F-statistic):           4.61e-08
Time:                        14:39:16   Log-Likelihood:                 1017.3
No. Observations:               30159   AIC:                            -2029.
Df Residuals:                   30156   BIC:                            -2004.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0349      0.002     20.815      0.000       0.032       0.038
pct_female     0.0336      0.012      2.866      0.004       0.011       0.057
ceoown         0.0971      0.019      5.140      0.000       0.060       0.134
==============================================================================
Omnibus:                    72310.238   Durbin-Watson:                   1.412
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       1772281663.706
Skew:                         -24.389   Prob(JB):                         0.00
Kurtosis:                    1189.580   Cond. No.                         14.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### Panel regression

In [23]:
execucomp_3 = execucomp_2.copy()
execucomp_3 = execucomp_3.set_index(['gvkey','fyear'])
PanelOLS.from_formula('roa~1+pct_female+ceoown+EntityEffects', \
    execucomp_3.dropna(subset=['roa','pct_female','ceoown'])).fit().summary

Dep. Variable:,roa,R-squared:,0.0002
Estimator:,PanelOLS,R-squared (Between):,-0.0014
No. Observations:,30159,R-squared (Within):,0.0002
Date:,"Thu, Aug 16 2018",R-squared (Overall):,0.0005
Time:,14:39:17,Log-likelihood,7272.7
Cov. Estimator:,Unadjusted,,
,,F-statistic:,2.6833
Entities:,3685,P-value,0.0684
Avg Obs:,8.1843,Distribution:,"F(2,26879)"
Min Obs:,0.0000,,
Max Obs:,26.000,F-statistic (robust):,2.6833
